# Plot images

Query img files by plate and site, then download TIFF, combine channels, and visualize

In [1]:
import polars as pl
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from sh import aws

In [2]:
index_path = "../1_snakemake/inputs/images/index.parquet"
meta_path = "../1_snakemake/inputs/metadata/metadata.parquet"
tiff_dir = "../1_snakemake/inputs/images/tiff"
png_dir = "../1_snakemake/inputs/images/png"
aws_img_path = "s3://cellpainting-gallery/cpg0037-oasis/axiom/images"

index = pl.read_parquet(index_path)
meta = pl.read_parquet(meta_path)

index = index.join(meta, on=["Metadata_Plate", "Metadata_Well"])

In [3]:
import numpy as np

def normalize(channel):
    print(channel.min())
    print(channel.max())
    return (channel - channel.min()) / (channel.max() - channel.min())

def normalize_quantile(channel):
    p99 = np.percentile(channel, 99.9)
    print(p99)
    return channel / p99

def normalize_channel(channel, channel_nm):
    if channel_nm == "DNA":
        p99 = 11000
    elif channel_nm == "ER":
        p99 = 13000
    elif channel_nm == "AGP":
        p99 = 8000
    elif channel_nm == "RNA":
        p99 = 7000
    elif channel_nm == "Mito":
        p99 = 11000

    return channel / p99

In [4]:
def get_tiffs(plate: str, well: str, site: int, batch: str, png_dir: str) -> list:
    png_path = f"{png_dir}/{plate}_{well}_{site}.png"
    
    channels = ["DNA", "ER", "AGP", "RNA", "Mito"]
    img_path = f"{tiff_dir}/{batch}/{plate}"
    tiffs = []

    for channel in channels:
        img_nm = (plot.filter(pl.col("Metadata_Plate") == plate)
                    .filter(pl.col("Metadata_Well") == well)
                    .filter(pl.col("Metadata_Site") == site)
                    .filter(pl.col("Channel") == channel)
        ).select("Filename").item()

        tiff_path = f"{img_path}/{img_nm}"

        if not os.path.exists(tiff_path):
            print(f"Downloading: {img_nm}")
            aws_img = f"{aws_img_path}/{batch}/images/{plate}/{img_nm}"
            aws("s3", "cp", aws_img, tiff_path)

        img = np.array(Image.open(tiff_path))
        tiffs.append(normalize_channel(img, channel))   

    return tiffs

In [5]:
from matplotlib.gridspec import GridSpec

def plot_tiffs(tiffs: list, png_path: str) -> None:
    rgb_image = np.zeros((tiffs[0].shape[0], tiffs[0].shape[1], 3))

    rgb_image[:, :, 0] = tiffs[4] + tiffs[2] + tiffs[3]  # Red component (for red, yellow, and purple)
    rgb_image[:, :, 1] = tiffs[1] + tiffs[2]  # Green component (for green and yellow)
    rgb_image[:, :, 2] = tiffs[0] + tiffs[3]  # Blue component (for blue and purple)

    rgb_image = np.clip(rgb_image, 0, 1)

    # Create a grid layout with GridSpec
    fig = plt.figure(figsize=(10, 12))
    gs = GridSpec(2, 5, height_ratios=[1, 4], hspace=0.01, wspace=0.01)

    # First row: Horizontal strip of smaller channel images
    channels = ["DNA", "ER", "AGP", "RNA", "Mito"]

    for i, (channel, title) in enumerate(zip(tiffs, channels)):
        ax = fig.add_subplot(gs[0, i])  # Place in the first row, ith column
        ax.imshow(channel, cmap="gray")
        ax.set_title(title, fontsize=10)
        ax.axis("off")

    # Second row: Combined RGB image
    ax_combined = fig.add_subplot(gs[1, :])  # Span all columns in the second row
    ax_combined.imshow(rgb_image)
    ax_combined.axis("off")

    plt.subplots_adjust(left=0.02, right=0.98, top=0.98, bottom=0.02)

    # Save the figure
    plt.savefig(png_path, dpi=300, bbox_inches="tight", pad_inches=0)
    plt.show()

## AR agonists and antagonists

In [6]:
strong_ag = ["OASIS1137", "OASIS1302", "OASIS1280", "OASIS1381", "OASIS1509", "OASIS1313", "OASIS1933"]
strong_antag = ["OASIS419", "OASIS1502", "OASIS940", "OASIS1098", "OASIS379", "OASIS1488", "OASIS794"]

In [7]:
ag_meta = meta.filter(pl.col("Metadata_OASIS_ID").is_in(strong_ag)).filter(pl.col("Metadata_Concentration") == 100)
antag_meta = meta.filter(pl.col("Metadata_OASIS_ID").is_in(strong_antag)).filter(pl.col("Metadata_Concentration") == 100)

In [25]:
# From this, define image metadata
plate = "plate_41002877"
well = "F09"
site = 6

plot = (index.filter(pl.col("Metadata_Plate") == plate)
             .filter(pl.col("Metadata_Well") == well))
plot.head()

Metadata_Batch,Metadata_Plate,Metadata_Well,Metadata_Site,Channel,Filename,Metadata_well_id,Metadata_source,Metadata_Concentration,Metadata_Compound,Metadata_compound_target,Metadata_compound_pathway,Metadata_compound_research_area,Metadata_compound_clinical_information,Metadata_row,Metadata_col,Metadata_mtt_lumi,Metadata_ldh_abs_signal,Metadata_ldh_abs_background,Metadata_ldh_abs,Metadata_mtt_normalized,Metadata_ldh_normalized,Metadata_mtt_ridge_norm,Metadata_ldh_ridge_norm,Metadata_OASIS_ID,Metadata_Perturbation,Metadata_Log10Conc,Metadata_Count_Cells
str,str,str,i64,str,str,str,str,f64,str,str,str,str,str,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,u32
"""prod_26""","""plate_41002877""","""F09""",1,"""Brightfield""","""r06c09f01p01-ch6sk1fk1fl1.tiff""","""assayworks_prod_26_p=plate_410…","""assayworks_prod_26""",100.0,"""Ketoconazole""","""Bacterial; Cytochrome P450; Fu…","""Anti-infection; GPCR/G Protein…","""Infection; Cancer""","""Launched""",5,8,12720.0,0.52,0.018,0.502,0.213182,0.064998,0.225204,0.157301,"""OASIS004""","""Ketoconazole_100.0""",3.817746,499
"""prod_26""","""plate_41002877""","""F09""",2,"""Brightfield""","""r06c09f02p01-ch6sk1fk1fl1.tiff""","""assayworks_prod_26_p=plate_410…","""assayworks_prod_26""",100.0,"""Ketoconazole""","""Bacterial; Cytochrome P450; Fu…","""Anti-infection; GPCR/G Protein…","""Infection; Cancer""","""Launched""",5,8,12720.0,0.52,0.018,0.502,0.213182,0.064998,0.225204,0.157301,"""OASIS004""","""Ketoconazole_100.0""",3.817746,499
"""prod_26""","""plate_41002877""","""F09""",3,"""Brightfield""","""r06c09f03p01-ch6sk1fk1fl1.tiff""","""assayworks_prod_26_p=plate_410…","""assayworks_prod_26""",100.0,"""Ketoconazole""","""Bacterial; Cytochrome P450; Fu…","""Anti-infection; GPCR/G Protein…","""Infection; Cancer""","""Launched""",5,8,12720.0,0.52,0.018,0.502,0.213182,0.064998,0.225204,0.157301,"""OASIS004""","""Ketoconazole_100.0""",3.817746,499
"""prod_26""","""plate_41002877""","""F09""",4,"""Brightfield""","""r06c09f04p01-ch6sk1fk1fl1.tiff""","""assayworks_prod_26_p=plate_410…","""assayworks_prod_26""",100.0,"""Ketoconazole""","""Bacterial; Cytochrome P450; Fu…","""Anti-infection; GPCR/G Protein…","""Infection; Cancer""","""Launched""",5,8,12720.0,0.52,0.018,0.502,0.213182,0.064998,0.225204,0.157301,"""OASIS004""","""Ketoconazole_100.0""",3.817746,499
"""prod_26""","""plate_41002877""","""F09""",5,"""Brightfield""","""r06c09f05p01-ch6sk1fk1fl1.tiff""","""assayworks_prod_26_p=plate_410…","""assayworks_prod_26""",100.0,"""Ketoconazole""","""Bacterial; Cytochrome P450; Fu…","""Anti-infection; GPCR/G Protein…","""Infection; Cancer""","""Launched""",5,8,12720.0,0.52,0.018,0.502,0.213182,0.064998,0.225204,0.157301,"""OASIS004""","""Ketoconazole_100.0""",3.817746,499


In [ ]:

batch = "prod_26"
png_path = f"{png_dir}/{plate}_{well}_{site}.png"

tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
plot_tiffs(tiffs, png_path)

In [36]:
dmso = ['DMSO_0.0_H22_plate_41002948',
        'DMSO_0.0_H11_plate_41002882',
        'DMSO_0.0_H24_plate_41002890',
        'DMSO_0.0_P11_plate_41002908',
        'DMSO_0.0_F04_plate_41002959',
        'DMSO_0.0_A20_plate_41002879',
        'DMSO_0.0_M04_plate_41002877',
        'DMSO_0.0_H10_plate_41002947',
        'DMSO_0.0_H02_plate_41002692',
        'DMSO_0.0_C04_plate_41002959']

higher = ['Enzalutamide_3.82_M11_plate_41002897',
        'Temsirolimus_1.43_A01_plate_41002900',
        'Benzethonium (chloride)_2.86_L22_plate_41002905',
        'Ximelagatran_3.82_A19_plate_41002878',
        'Alfacalcidol_3.82_O23_plate_41002877',
        'Puromycin aminonucleoside_3.34_M06_plate_41002892',
        'Hycanthone_3.34_G16_plate_41002957',
        'Alectinib (Hydrochloride)_3.34_K07_plate_41002956',
        'Rilpivirine_3.82_B17_plate_41002896',
        'Fesoterodine_3.82_A13_plate_41002878']

lower = ['Nevirapine_3.34_N14_plate_41002956',
        'Capsaicin_3.34_P14_plate_41002701',
        'Ezetimibe_3.82_J18_plate_41002946',
        'Melatonin_3.82_D16_plate_41002692',
        'Safinamide (mesylate)_3.82_L13_plate_41002693',
        'Atenolol_2.86_G22_plate_41002951',
        '2,7-Dihydroxynaphthalene_3.82_M09_plate_41002877',
        'Linagliptin_2.86_J02_plate_41002904',
        'Alpidem_3.82_O17_plate_41002877',
        '5-Azacytidine_2.86_I02_plate_41002951']

In [ ]:
for pert in dmso:
    well = pert.split("_")[2]
    plate = pert.split("_")[4]

    plot = (index.filter(pl.col("Metadata_Plate").str.contains(plate))
             .filter(pl.col("Metadata_Well") == well))
    
    batch = plot.select(pl.col("Metadata_Batch")).to_series().to_list()[0]
    plate = plot.select(pl.col("Metadata_Plate")).to_series().to_list()[0]
    site = 6

    png_path = f"{png_dir}/dmso/{plate}_{well}_{site}.png"

    tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
    plot_tiffs(tiffs, png_path)

In [ ]:
for row in antag_meta.iter_rows(named=True): 
    well = row["Metadata_Well"]
    plate = row["Metadata_Plate"]
    batch = row["Metadata_source"]
    batch = batch.replace("assayworks_", "")
    exp = row["Metadata_Compound"]

    site = 6

    plot = (index.filter(pl.col("Metadata_Plate").str.contains(plate))
            .filter(pl.col("Metadata_Well") == well))

    png_path = f"{png_dir}/AR_antagonist/{plate}_{well}_{site}_{exp}.png"

    tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
    plot_tiffs(tiffs, png_path)

## Cell count increases

In [10]:
cmpds = ["Baloxavir marboxil", "Ceritinib", "Alectinib (Hydrochloride)"]
concs = [100, 20]

cc_meta = meta.filter(pl.col("Metadata_Compound").is_in(cmpds)).filter(pl.col("Metadata_Concentration").is_in(concs))

In [ ]:
for row in cc_meta.iter_rows(named=True):
    well = row["Metadata_Well"]
    plate = row["Metadata_Plate"]
    batch = row["Metadata_source"]
    batch = batch.replace("assayworks_", "")
    exp = row["Metadata_Compound"]

    site = 6

    plot = (index.filter(pl.col("Metadata_Plate").str.contains(plate))
            .filter(pl.col("Metadata_Well") == well))

    png_path = f"{png_dir}/cc_increase/{plate}_{well}_{site}_{exp}.png"

    tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
    plot_tiffs(tiffs, png_path)

## Alectinib (crazy phenotype of firework nuclei)

In [ ]:
alectinib_meta = meta.filter(pl.col("Metadata_Compound") == "Alectinib (Hydrochloride)")

for row in alectinib_meta.iter_rows(named=True):
    well = row["Metadata_Well"]
    plate = row["Metadata_Plate"]
    batch = row["Metadata_source"]
    batch = batch.replace("assayworks_", "")
    exp = row["Metadata_Compound"]
    conc = row["Metadata_Concentration"]
    conc = float(f"{conc:.1g}")

    site = 6

    plot = (index.filter(pl.col("Metadata_Plate").str.contains(plate))
            .filter(pl.col("Metadata_Well") == well))

    png_path = f"{png_dir}/alectinib/{plate}_{well}_{site}_{conc}_{exp}.png"

    tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
    plot_tiffs(tiffs, png_path)

We see the fragmented nuclei at both of the highest dose (20.0) and one of the two second-highest (7.0). A few very small DNA fragments at the third highest (2.0).

In [6]:
cmpd_meta = meta.filter(pl.col("Metadata_Compound") == "Colchicine")

for row in cmpd_meta.iter_rows(named=True):
    well = row["Metadata_Well"]
    plate = row["Metadata_Plate"]
    batch = row["Metadata_source"]
    batch = batch.replace("assayworks_", "")
    exp = row["Metadata_Compound"]
    conc = row["Metadata_Concentration"]
    conc = float(f"{conc:.1g}")

    site = 6

    plot = (index.filter(pl.col("Metadata_Plate").str.contains(plate))
            .filter(pl.col("Metadata_Well") == well))

    png_path = f"{png_dir}/colchicine/{exp}_{conc}_{well}_{plate}_{site}.png"

    tiffs = get_tiffs(plate, well, site, batch, tiff_dir)
    plot_tiffs(tiffs, png_path)